In [38]:
# Importing libraries

import os
import torch
import argparse
import numpy as np 
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import *
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [39]:
# importing models
import torchvision.models as models

In [12]:
ls

face_mask_datasets@  face_mask.ipynb  Untitled0.ipynb


In [11]:
cd drive/My Drive/Neurodynamics/Pytorch-haxora/Face Mask Prediction

/content/drive/My Drive/Neurodynamics/Pytorch-haxora/Face Mask Prediction


In [40]:
transform_train = transforms.Compose([transforms.Resize((256,256)),
                                    transforms.RandomCrop((224,224)), # resizing image size to (224,224)
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
                                  ])

transform_val = transforms.Compose([transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
                                ])

In [24]:
ls

face_mask_dataset_kaggle/


In [41]:
dir = 'face_mask_dataset_kaggle' # datasets path

In [42]:
dataset = ImageFolder(root = dir, transform=transform_train)
trainset, valset = torch.utils.data.random_split(dataset, [3033, 800])

In [43]:
val_set

In [44]:
# Loading images
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers = 2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False, num_workers = 2)

In [45]:
# Here i have used resnet101 and its last layer(output layer) should be modeified as per requirement
model = models.resnet101(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
# freezing the models weights
for param in model.parameters():
  param.requires_grad = False

In [47]:
# changing last layer (output layer) of the model
model.fc=nn.Linear(2048,2)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [48]:
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler=StepLR(optimizer,step_size=3)
criterion=nn.CrossEntropyLoss()
criterion.cuda()

CrossEntropyLoss()

In [49]:
# training the model

def train(epoch):
	print('\nEpoch: %d' % epoch)
	scheduler.step()
	model.train()
	for batch_idx,(img,label) in enumerate(trainloader):
		image=Variable(img.cuda())
		label=Variable(label.cuda())
		optimizer.zero_grad()
		out=model(image)
		loss=criterion(out,label)
		loss.backward()
		optimizer.step()
		print("Epoch:%d [%d|%d] loss:%f" %(epoch,batch_idx,len(trainloader),loss.mean()))

def val(epoch):
	print("\nValidation Epoch: %d" %epoch)
	model.eval()
	total=0
	correct=0
	with torch.no_grad():
		for batch_idx,(img,label) in enumerate(valloader):
			image=Variable(img.cuda())
			label=Variable(label.cuda())
			out=model(image)
			_,predicted=torch.max(out.data,1)
			total+=image.size(0)
			correct+=predicted.data.eq(label.data).cpu().sum()
	print("Acc: %f "% ((1.0*correct.numpy())/total))

In [53]:
epochs = 1
for epoch in range(epochs):
	train(epoch)
	val(epoch)
# torch.save(model.state_dict(),'face_mask_model.pth')


Epoch: 0


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch:0 [0|48] loss:0.139014
Epoch:0 [1|48] loss:0.130653
Epoch:0 [2|48] loss:0.133815
Epoch:0 [3|48] loss:0.159844
Epoch:0 [4|48] loss:0.154903
Epoch:0 [5|48] loss:0.165569
Epoch:0 [6|48] loss:0.136794
Epoch:0 [7|48] loss:0.152254
Epoch:0 [8|48] loss:0.144812
Epoch:0 [9|48] loss:0.102618
Epoch:0 [10|48] loss:0.122683
Epoch:0 [11|48] loss:0.128452
Epoch:0 [12|48] loss:0.130135
Epoch:0 [13|48] loss:0.109580
Epoch:0 [14|48] loss:0.135580
Epoch:0 [15|48] loss:0.151093
Epoch:0 [16|48] loss:0.145477
Epoch:0 [17|48] loss:0.143649
Epoch:0 [18|48] loss:0.129534
Epoch:0 [19|48] loss:0.133313
Epoch:0 [20|48] loss:0.155339
Epoch:0 [21|48] loss:0.158683
Epoch:0 [22|48] loss:0.163362
Epoch:0 [23|48] loss:0.200523
Epoch:0 [24|48] loss:0.113432
Epoch:0 [25|48] loss:0.143094
Epoch:0 [26|48] loss:0.125926
Epoch:0 [27|48] loss:0.155798
Epoch:0 [28|48] loss:0.153627
Epoch:0 [29|48] loss:0.145039
Epoch:0 [30|48] loss:0.131999
Epoch:0 [31|48] loss:0.121459
Epoch:0 [32|48] loss:0.146034
Epoch:0 [33|48] loss

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch:0 [42|48] loss:0.149739
Epoch:0 [43|48] loss:0.165341
Epoch:0 [44|48] loss:0.127752
Epoch:0 [45|48] loss:0.156104
Epoch:0 [46|48] loss:0.112786
Epoch:0 [47|48] loss:0.250207

Validation Epoch: 0
Acc: 0.972500 


In [64]:
# Saving the trained model
torch.save(model.state_dict(),'face_mask_model.pth')

In [71]:
# Loading the saved model

# model = Net()
model.load_state_dict(torch.load('face_mask_model.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 